**Проект e-commerce:** 
**Продакт-менеджер Василий попросил вас проанализировать совершенные покупки и ответить на следующие вопросы:**

1. Сколько у нас пользователей, которые совершили покупку только один раз? (7 баллов) 

2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)? (5 баллов)

3. По каждому товару определить, в какой день недели товар чаще всего покупается. (5 баллов)

4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? Не стоит забывать, что внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть. (8 баллов) 

5. Используя pandas, проведи когортный анализ пользователей. В период с января по декабрь выяви когорту с самым высоким retention на 3й месяц. Описание подхода можно найти тут. (10 баллов)

6. Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построй RFM-сегментацию пользователей, чтобы качественно оценить свою аудиторию. В кластеризации можешь выбрать следующие метрики: R - время от последней покупки пользователя до текущей даты, F - суммарное количество покупок у пользователя за всё время, M - сумма покупок за всё время. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, frequency от 2 до 5 заказов в неделю, monetary от 1780 до 3560 рублей в неделю. Описание подхода можно найти тут. (35 баллов)

**Для решения задачи проведи предварительное исследование данных и сформулируй, что должно считаться покупкой. Обосновать свой выбор ты можешь с помощью фактов оплат, статусов заказов и других имеющихся данных.**

**Файлы:**

**olist_customers_dataset.csv — таблица с уникальными идентификаторами пользователей**

 - customer_id — позаказный идентификатор пользователя

 - customer_unique_id —  уникальный идентификатор пользователя  (аналог номера паспорта)

 - customer_zip_code_prefix —  почтовый индекс пользователя

 - customer_city —  город доставки пользователя

 - customer_state —  штат доставки пользователя

**olist_orders_dataset.csv —  таблица заказов**

 - order_id —  уникальный идентификатор заказа (номер чека)
 - customer_id —  позаказный идентификатор пользователя
 - order_status —  статус заказа
 - order_purchase_timestamp —  время создания заказа
 - order_approved_at —  время подтверждения оплаты заказа
 - order_delivered_carrier_date —  время передачи заказа в логистическую службу
 - order_delivered_customer_date —  время доставки заказа
 - order_estimated_delivery_date —  обещанная дата доставки
 
**olist_order_items_dataset.csv —  товарные позиции, входящие в заказы**
 - order_id —  уникальный идентификатор заказа (номер чека)
 - order_item_id —  идентификатор товара внутри одного заказа
 - product_id —  ид товара (аналог штрихкода)
 - seller_id — ид производителя товара
 - shipping_limit_date —  максимальная дата доставки продавцом для передачи заказа партнеру по логистике
 - price —  цена за единицу товара
 - freight_value —  вес товара

— Пример структуры данных можно визуализировать по order_id == 00143d0f86d6fbd9f9b38ab440ac16f5

Уникальные статусы заказов в таблице olist_orders_dataset:

 - created —  создан
 - approved —  подтверждён
 - invoiced —  выставлен счёт
 - processing —  в процессе сборки заказа
 - shipped —  отгружен со склада
 - delivered —  доставлен пользователю
 - unavailable —  недоступен
 - canceled —  отменён

In [9]:
import pandas as pd
import numpy as np
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
import os

%matplotlib inline 

In [10]:
# Открываем ДФ с данными о покупателях

olist_customers_dataset = pd.read_csv('olist_customers_dataset.csv')

In [11]:
# Проверяем ДФ с данными о покупателях

olist_customers_dataset.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [12]:
# Смотрим инфо о ДФ

olist_customers_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [13]:
olist_customers_dataset.customer_id.nunique()             

99441

In [14]:
olist_customers_dataset.customer_unique_id.nunique()

96096

In [15]:
olist_customers_dataset.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
99436    False
99437    False
99438    False
99439    False
99440    False
Length: 99441, dtype: bool

**В ДФ olist_customers_dataset:**

- отсуствуют NaN значения
- количество уникальных клиентов 96096

In [16]:
# Открываем ДФ с данными о заказах покупателей

olist_orders_dataset = pd.read_csv('olist_orders_dataset.csv', parse_dates=[3,4,5,6,7])

In [17]:
olist_orders_dataset.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


In [18]:
olist_orders_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB


In [19]:
olist_orders_dataset.customer_id.nunique()   

99441

In [20]:
olist_orders_dataset.order_id.nunique()

99441

In [21]:
olist_orders_dataset.order_status.unique()

array(['delivered', 'invoiced', 'shipped', 'processing', 'unavailable',
       'canceled', 'created', 'approved'], dtype=object)

In [22]:
olist_orders_dataset.groupby('order_status').agg({'order_id':'count'})

,order_id
order_status,
approved,2
canceled,625
created,5
delivered,96478
invoiced,314
processing,301
shipped,1107
unavailable,609


In [23]:
olist_orders_dataset.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
99436    False
99437    False
99438    False
99439    False
99440    False
Length: 99441, dtype: bool

**В следующих столбцах ДФ olist_orders_dataset присуствуют NaN значения:**
 - order_approved_at   (160 строк)
 - order_delivered_carrier_date   (1783 строк)
 - order_delivered_customer_date   (2965 строк)
 
 
 **Количество уникальных значений столбца customer_id в ДФ olist_customers_dataset и ДФ olist_orders_dataset совпадает**

In [24]:
# Открываем ДФ с данными о товарах в заказах покупателей

olist_order_items_dataset = pd.read_csv('olist_order_items_dataset.csv', parse_dates = [5])

In [25]:
olist_order_items_dataset.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [26]:
olist_order_items_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  object 
 6   freight_value        112650 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 6.0+ MB


In [27]:
# Смотрим количество уникальных order_id

olist_order_items_dataset.order_id.nunique()

98666

In [28]:
olist_order_items_dataset.product_id.nunique()

32951

In [29]:
olist_order_items_dataset.duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
112645    False
112646    False
112647    False
112648    False
112649    False
Length: 112650, dtype: bool

В ДФ olist_order_items_dataset:

- отсуствуют NaN значения
- общее количество строк больше, чем в olist_customers_dataset и olist_orders_dataset (112 650 против 99 441), следовательно в некоторых заказах было несколько предметов.
- В ДФ olist_orders_dataset количество уникальных значений столбца order_id больше, чем в olist_order_items_dataset (99441 против 98 666). Это значит, что по части сделок нет информации о содержимом заказа.

**Промежуточный вывод:** 
к покупке следует отнести событие, имеющее оличительный статус заказа и временной маркер, поэтому в качестве **покупки в данной задаче** определим событие "подтверждение заказа" (**статус заказа approved**), так как для события "подтверждение заказа" фиксируется время (**временной маркер order_approved_at**) и этот событие в наиболее полной мере отражает намерение клиента получить желаемый товар, путем оплаты заказа. Другие события (shipped и delivered), время которых фиксируется (order_delivered_carrier_date и order_delivered_customer_date соотвественно), скорее отражают качество работы логистической службы, отвечающей за наличие товара на складе, отправку и доставку товара покупателю. Создание заказа клиентом также не может в полной мере отразить факт покупки, так как создание заказа может произойти по ошибке или в мошеннических целях (для имитирования целевых действий).

**1. Сколько у нас пользователей, которые совершили покупку только один раз? (7 баллов)**

In [30]:
# Сджойним данные о клиентах и сделках по колонке customer_id

customers_orders = olist_customers_dataset.merge(olist_orders_dataset, how = 'outer', on = 'customer_id')

In [31]:
customers_orders.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15


In [32]:
# В получившемся ДФ проверям количество уникальных значений колонки customer_id

customers_orders.customer_id.nunique()

99441

**Промежуточный вывод:** количество уникальных значений customer_id после джойна не изменилось, делаем вывод о полном соотвествии содержимого столбцов customer_id в olist_customers_dataset и olist_orders_dataset

In [33]:
# Часть клиентов, которые совершили покупку

customers_orders.query('order_approved_at.isnull() and order_status == "delivered"')

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
8970,0bf35cac6cc7327065da879e2d90fae8,c4c0011e639bdbcf26059ddc38bd3c18,13225,varzea paulista,SP,d77031d6a3c8a52f019764e68f211c69,delivered,2017-02-18 11:04:19,NaT,2017-02-23 07:23:36,2017-03-02 16:15:23,2017-03-22
13890,1e101e0daffaddce8159d25a8e53f2b2,c8822fce1d0bfa7ddf0da24fff947172,27945,macae,RJ,12a95a3c06dbaec84bcfb0e2da5d228a,delivered,2017-02-17 13:05:55,NaT,2017-02-22 11:23:11,2017-03-02 11:09:19,2017-03-20
20863,d5de688c321096d15508faae67a27051,d49f3dae6bad25d05160fc17aca5942d,36400,conselheiro lafaiete,MG,7002a78c79c519ac54022d4f8a65e6e8,delivered,2017-01-19 22:26:59,NaT,2017-01-27 11:08:05,2017-02-06 14:22:19,2017-03-16
22703,07a2a7e0f63fd8cb757ed77d4245623c,79af1bbf230a2630487975aa5d7d6220,37660,paraisopolis,MG,51eb2eebd5d76a24625b31c33dd41449,delivered,2017-02-18 15:52:27,NaT,2017-02-23 03:09:14,2017-03-07 13:57:47,2017-03-29
27441,68d081753ad4fe22fc4d410a9eb1ca01,2e0a2166aa23da2472c6a60c4af6f7a6,3573,sao paulo,SP,d69e5d356402adc8cf17e08b5033acfb,delivered,2017-02-19 01:28:47,NaT,2017-02-23 03:11:48,2017-03-02 03:41:58,2017-03-27
39286,d85919cb3c0529589c6fa617f5f43281,c094ac95fcd52f821809ec232a7a6956,95795,sao vendelino,RS,3c0b8706b065f9919d0505d3b3343881,delivered,2017-02-17 15:53:27,NaT,2017-02-22 11:31:30,2017-03-03 11:47:47,2017-03-23
41493,74bebaf46603f9340e3b50c6b086f992,f79be7c08dd24b72d34634f1b89333a4,65110,sao jose de ribamar,MA,2babbb4b15e6d2dfe95e2de765c97bce,delivered,2017-02-18 17:15:03,NaT,2017-02-22 11:23:11,2017-03-03 18:43:43,2017-03-31
44050,684cb238dc5b5d6366244e0e0776b450,6ff8b0d7b35d5c945633b8d60165691b,11030,santos,SP,c1d4211b3dae76144deccd6c74144a88,delivered,2017-01-19 12:48:08,NaT,2017-01-25 14:56:50,2017-01-30 18:16:01,2017-03-01
52849,a3d3c38e58b9d2dfb9207cab690b6310,5a4fa4919cbf2b049e72be460a380e5b,35620,abaete,MG,2eecb0d85f281280f79fa00f9cec1a95,delivered,2017-02-17 17:21:55,NaT,2017-02-22 11:42:51,2017-03-03 12:16:03,2017-03-20
60039,2127dc6603ac33544953ef05ec155771,8a9a08c7ca8900a200d83cf838a07e0b,6708,cotia,SP,e04abd8149ef81b95221e88f6ed9ab6a,delivered,2017-02-18 14:40:00,NaT,2017-02-23 12:04:47,2017-03-01 13:25:33,2017-03-17


In [34]:
# Находим в customers_orders количество покупателей, которые совершили покупку только один раз
# Добавляем условие order_status == "delivered" так как некоторые пользователи получили товар, однако временной маркер order_approved_at у них отсуствует  

customers_orders.query('not(order_approved_at.isnull()) or order_status == "delivered"')\
                .groupby('customer_unique_id', as_index = False)\
                .agg(orders_per_customer=('order_id', 'count'))\
                .groupby('orders_per_customer', as_index = False)\
                .agg(count=('customer_unique_id', 'count'))

,orders_per_customer,count
0,1,93061
1,2,2709
2,3,191
3,4,30
4,5,8
5,6,6
6,7,3
7,9,1
8,17,1


**Ответ на задание №1:** 93 049 покупателей совершили только одну покупку

**2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)? (5 баллов)**

Необходимо проверить путь заказа от создания до доставки для выявления причин, по которым часть заказов не было доставлено. Для этого обратимся к временным маркерам, составим из событий для этих маркеров воронку и на каждом этапе воронки проверим статусы заказов.

С учетом имеющихся временных маркеров выделим следующие этапы воронки:

- создание заказа (order_purchase_timestamp)
- подтверждение оплаты заказа (order_approved_at)
- передача заказа в логистическую службу (order_delivered_carrier_date)
- доставка заказа покупателю (order_delivered_customer_date)

In [35]:
# Находим временной интервал, для которого будем считать средние значения

customers_orders.order_delivered_customer_date.dt.to_period('M').max() - customers_orders.order_purchase_timestamp.dt.to_period('M').min()

<25 * MonthEnds>

In [36]:
# Найдем количество месяцев другим способом

max_date = customers_orders.order_delivered_customer_date.dt.to_period('d').max()
min_date = customers_orders.order_purchase_timestamp.dt.to_period('d').min()

def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

months = diff_month(max_date, min_date)
months

25

Временной интервал составляет 25 месяцев

**Этап "Создание Заказа - Оплата"**

In [37]:
# Выделим заказы которые были созданы, но неоплачены, сгруппируем их по статусу
# Также отфильтруем заказы, которые в итоге были доставлены (order_status ='delivered')

unpaid = customers_orders.query('not(order_purchase_timestamp.isnull()) and order_approved_at.isnull() and order_status!="delivered"')\
                .groupby('order_status', as_index = False)\
                .agg(count=('customer_unique_id', 'count'))
unpaid

,order_status,count
0,canceled,141
1,created,5


Статусов invoiced на этапе "Создание заказа-Оплата" нет, значит статус invoiced не связан со сделкой между покупателем и онлайн-ритейлером(в противном случае хотя бы некоторые заказы остались бы "висеть" в системе с неоплаченными счетами, такое часто бывает), причиной же является иное событие, пояснения ниже

**Этап "Оплата - Передача в доставку"**

In [38]:
# Выделим заказы которые были оплачены, но не отгружены, сгруппируем их по статусу
# Также отфильтруем заказы, которые в итоге были доставлены (order_status ='delivered')

unshipped = customers_orders.query('not(order_approved_at.isnull()) and order_delivered_carrier_date.isnull() and order_status!="delivered"')\
                .groupby('order_status', as_index = False)\
                .agg(count=('customer_unique_id', 'count'))
unshipped

,order_status,count
0,approved,2
1,canceled,409
2,invoiced,314
3,processing,301
4,unavailable,609


Наблюдаем 314 статусов invoiced на этапе "Оплата-Передача в доставку"

**Промежуточный вывод:** так как статус approved и временной маркер order_aproved_at связаны с одним и тем же событием (с оплатой), а на этапе "Создание заказа - Оплата" не наблюдается статусов invoiced, то можно сделать вывод, что этап invoiced проходят покупки, которые приходят из-за границы от компаний-партнеров местного онлайн-ритейлера. При этом в процессе выполнения заказа для провоза через границу между онлайн-ритейлером и компанией-партнером согласовывается и оформляется инвойс. Исходя из этой логики часть заказов были оплачены покупателями, но онлайн-ритейлер и зарубежная компания-партнер не смогли завершить оформление инвойса (или оформили некоррекно), заказы "зависли" у компаний-партнеров и не были переданы в доставку (временной маркер order_delivered_carrier_date для всех статусов invoiced не определен), покупатели почему-то решили не отменять заказ, в итоге в базе данных онлайн-ритейлера остался статус заказа invoiced.

**Этап "Передача в доставку - Доставка покупателю"**

In [39]:
# Выделим заказы которые были переданы в логистическую службу, но не были доставлены, сгруппируем их по статусу
# Также отфильтруем заказы, которые в итоге были доставлены (order_status ='delivered')

undelivered = customers_orders.query('not(order_delivered_carrier_date.isnull()) and order_delivered_customer_date.isnull() and order_status != "delivered"')\
                .groupby('order_status', as_index = False)\
                .agg(count=('customer_unique_id', 'count'))
undelivered

,order_status,count
0,canceled,69
1,shipped,1107


Найдем средние значения количества заказов в месяц, которые не доставляются по различным причинам на каждом шаге воронки

In [40]:
unpaid['mean_count'] = unpaid['count']/months
unpaid

,order_status,count,mean_count
0,canceled,141,5.64
1,created,5,0.20


In [41]:
unshipped['mean_count'] = unshipped['count']/months
unshipped

,order_status,count,mean_count
0,approved,2,0.08
1,canceled,409,16.36
2,invoiced,314,12.56
3,processing,301,12.04
4,unavailable,609,24.36


In [42]:
undelivered['mean_count'] = undelivered['count']/months
undelivered

,order_status,count,mean_count
0,canceled,69,2.76
1,shipped,1107,44.28


**Ответ на задание №2:**

**Этап "Создание заказа - Оплата"**
- Недоставленые заказы из-за отмены заказа сразу после создания: **5.64 зак/мес**
- Недоставленые заказы из-за создания заказа и без его дальнейшей отмены: **0,20 зак/мес**

**Этап "Оплата-Передача в доставку"**
- Недоставленые заказы при подтвержденной оплате: **0.08 зак/мес**
- Недоставленые заказы из-за отмены заказа сразу после оплаты: **16.36 зак/мес**
- Недоставленые заказы из-за сложностей на этапе оформления инвойса с зарубежной компанией-партнером: **12.56 зак/мес**
- Недоставленые заказы из-за сложностей на этапе подготовки заказа к отгрузке: **12,04 зак/мес**

**Этап "Передача в доставку - Доставка покупателю"**
- Недоставленые заказы из-за отмены оплаченного заказа, который уже был передан в доставку: **2.76 зак/мес**
- Отгруженные со склада заказы, но недоставленые покупателям: **44.28 зак/мес**

**3. По каждому товару определить, в какой день недели товар чаще всего покупается. (5 баллов)**

In [43]:
# Сджойним данные о сделках и товарах по колонке order_id

order_items = olist_order_items_dataset.merge(olist_orders_dataset, how = 'outer', on = 'order_id')

In [44]:
order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,00010242fe8c5a6d1ba2dd792cb16214,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29
1,00018f77f2f0320c557190d7a144bdd3,1.0,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15
2,000229ec398224ef6ca0657da4fc703e,1.0,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05
3,00024acbcdf0a6daa1e931b038114c75,1.0,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20
4,00042b26cf59d7ce69dfabb4e55b4fd9,1.0,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17


In [45]:
order_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113425 entries, 0 to 113424
Data columns (total 14 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       113425 non-null  object        
 1   order_item_id                  112650 non-null  float64       
 2   product_id                     112650 non-null  object        
 3   seller_id                      112650 non-null  object        
 4   shipping_limit_date            112650 non-null  object        
 5   price                          112650 non-null  object        
 6   freight_value                  112650 non-null  float64       
 7   customer_id                    113425 non-null  object        
 8   order_status                   113425 non-null  object        
 9   order_purchase_timestamp       113425 non-null  datetime64[ns]
 10  order_approved_at              113264 non-null  datetime64[ns]
 11  

In [46]:
# Найдем количество строк где product_id определен, а order_approved_at нет

order_items.query('not(product_id.isnull()) and order_approved_at.isnull()').shape

(15, 14)

In [47]:
# найдем среднее значение разности между временем создания заказа и его оплаты, в целях возможной замены временного маркера order_approved_at

(order_items['order_approved_at'] - order_items['order_purchase_timestamp']).mean()

Timedelta('0 days 10:39:48.695287116')

In [48]:
# найдем среднее значение разности между временем оплаты заказа и его передачи в доставку, , в целях возможной замены временного маркера order_approved_at

(order_items['order_delivered_carrier_date'] - order_items['order_approved_at']).mean()

Timedelta('2 days 20:25:34.971070153')

**Промежуточный вывод:** так как разница между order_approved_at и order_purchase_timestamp, order_delivered_carrier_date и order_approved_at велики, а количество строк, где product_id определен, а order_approved_at нет, мало, то присваивать неопределенному временному маркеру order_approved_at какое-то иное значение нецелесообразно.

Избавимся от строк, где product_id определен, а order_approved_at нет, так как корректно определить день недели в который была совершена покупка не представляется возможным.

In [49]:
# Используем .dropna, чтобы избавиться от Null значений столбца product_id и order_approved_at ()

purchased_items = order_items.dropna(subset =['product_id','order_approved_at'])

In [50]:
purchased_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112635 entries, 0 to 112649
Data columns (total 14 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       112635 non-null  object        
 1   order_item_id                  112635 non-null  float64       
 2   product_id                     112635 non-null  object        
 3   seller_id                      112635 non-null  object        
 4   shipping_limit_date            112635 non-null  object        
 5   price                          112635 non-null  object        
 6   freight_value                  112635 non-null  float64       
 7   customer_id                    112635 non-null  object        
 8   order_status                   112635 non-null  object        
 9   order_purchase_timestamp       112635 non-null  datetime64[ns]
 10  order_approved_at              112635 non-null  datetime64[ns]
 11  

In [51]:
# Назначаем временной маркер order_approved_at индексом ДФ

purchased_items_time_index = purchased_items.set_index('order_approved_at')

In [52]:
# Создаем дополнительную колонку с названием дня недели

purchased_items_time_index = purchased_items_time_index.assign(weekday = lambda x: pd.to_datetime(x.index).strftime('%A'))

In [53]:
purchased_items_time_index.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,weekday
order_approved_at,,,,,,,,,,,,,,
2017-09-13 09:45:35,00010242fe8c5a6d1ba2dd792cb16214,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29,Wednesday
2017-04-26 11:05:13,00018f77f2f0320c557190d7a144bdd3,1.0,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15,Wednesday
2018-01-14 14:48:30,000229ec398224ef6ca0657da4fc703e,1.0,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,Sunday
2018-08-08 10:10:18,00024acbcdf0a6daa1e931b038114c75,1.0,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20,Wednesday
2017-02-04 14:10:13,00042b26cf59d7ce69dfabb4e55b4fd9,1.0,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17,Saturday


In [54]:
# Группируем по product_id и weekday, подсчитываем количество и применим функцию Pivot

item_wd_freq = purchased_items_time_index.groupby(['product_id', 'weekday'], as_index=False)\
                          .agg({'order_status':'count'})\
                          .pivot(index='product_id', columns='weekday', values='order_status')\
                          .reset_index()
item_wd_freq

weekday,product_id,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,00066f42aeeb9f3007548bb9d3f33c38,NaN,NaN,NaN,1.0,NaN,NaN,NaN
1,00088930e925c41fd95ebfe695fd2655,NaN,NaN,NaN,NaN,NaN,1.0,NaN
2,0009406fd7479715e4bef61dd91f2462,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,000b8f95fcb9e0096488278317764d19,1.0,NaN,NaN,NaN,NaN,NaN,1.0
4,000d9be29b5207b54e86aa1b1ac54872,NaN,NaN,NaN,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...
32943,fff6177642830a9a94a0f2cba5e476d1,NaN,1.0,1.0,NaN,NaN,NaN,NaN
32944,fff81cc3158d2725c0655ab9ba0f712c,NaN,1.0,NaN,NaN,NaN,NaN,NaN
32945,fff9553ac224cec9d15d49f5a263411f,NaN,NaN,1.0,NaN,NaN,NaN,NaN
32946,fffdb2d0ec8d6a61f0a0a0db3f25b441,1.0,NaN,NaN,NaN,1.0,3.0,NaN


In [55]:
# По каждому товару определим, максимальную частоту

item_wd_freq['max_wd_freq'] = item_wd_freq[['Friday','Monday','Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']].max(axis=1)
item_wd_freq.head(10)

weekday,product_id,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,max_wd_freq
0,00066f42aeeb9f3007548bb9d3f33c38,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0
1,00088930e925c41fd95ebfe695fd2655,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
2,0009406fd7479715e4bef61dd91f2462,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,000b8f95fcb9e0096488278317764d19,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0
4,000d9be29b5207b54e86aa1b1ac54872,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
5,0011c512eb256aa0dbbb544d8dffcf6e,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0
6,00126f27c813603687e6ce486d909d01,NaN,NaN,NaN,2.0,NaN,NaN,NaN,2.0
7,001795ec6f1b187d37335e1c4704762e,1.0,3.0,2.0,NaN,NaN,2.0,1.0,3.0
8,001b237c0e9bb435f2e54071129237e9,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
9,001b72dfd63e9833e8c02742adf472e3,2.0,NaN,3.0,1.0,4.0,2.0,2.0,4.0


In [56]:
item_wd_freq.product_id.nunique()

32948

In [57]:
olist_order_items_dataset.product_id.nunique()

32951

Для трех product_id не удалось определить, в какой день недели совершалась покупка

**Ответ на задание №3:** смотри значения в столбце max_wd_freq в ДФ item_wd_freq

**4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? Не стоит забывать, что внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть. (8 баллов)**

In [99]:
# Исключаем непокупки

orders_per_customer = customers_orders.query('not(order_approved_at.isnull())')

In [100]:
# Переводим order_approved_at

orders_per_customer['order_approved_at'] = orders_per_customer.order_approved_at.dt.to_period('M')

C:\Users\repin\AppData\Local\Temp\ipykernel_14524\47731422.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_per_customer['order_approved_at'] = orders_per_customer.order_approved_at.dt.to_period('M')


In [103]:
# Группируем по пользователям и месяцам

orders_per_customer_by_mth = orders_per_customer.groupby(['customer_unique_id','order_approved_at'], as_index=False).agg(oders_per_customer=('order_id','count'))

In [106]:
# Формируем колонку с количеством дней в каждом месяце

orders_per_customer_by_mth['weeks_in_month'] = orders_per_customer_by_mth['order_approved_at'].dt.days_in_month/7

In [110]:
# Создаем колонку с искомой метрикой

orders_per_customer_by_mth['oders_per_week'] = orders_per_customer_by_mth['oders_per_customer']/orders_per_customer_by_mth['weeks_in_month']
orders_per_customer_by_mth

,customer_unique_id,order_approved_at,oders_per_customer,weeks_in_month,oders_per_week
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,1,4.428571,0.225806
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05,1,4.428571,0.225806
2,0000f46a3911fa3c0805444483337064,2017-03,1,4.428571,0.225806
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10,1,4.428571,0.225806
4,0004aac84e0df4da2b147fca70cf8255,2017-11,1,4.285714,0.233333
...,...,...,...,...,...
97907,fffcf5a5ff07b0908bd4e2dbc735a684,2017-06,1,4.285714,0.233333
97908,fffea47cd6d3cc0a88bd621562a9d061,2017-12,1,4.428571,0.225806
97909,ffff371b4d645b6ecea244b27531430a,2017-02,1,4.000000,0.250000
97910,ffff5962728ec6157033ef9805bacc48,2018-05,1,4.428571,0.225806


**Ответ на задание №4:** смотри значения в столбце oders_per_week в ДФ orders_per_customer_by_mth

**Задания №5 и №6 в работе, будут запушены позднее**